<a href="https://colab.research.google.com/github/Domdec32/IMLO/blob/main/IMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torch.nn.functional as F
from torch.utils.data import ConcatDataset
from torch.utils.data import TensorDataset

In [11]:
ImageWidth = 200
ImageHeight = 200
batch_size = 32
LearningRate = 0.001
NumberofEpochs = 30
batch_size = 64

In [12]:
transform = transforms.Compose([
    transforms.Resize((ImageHeight, ImageWidth)),
    transforms.ToTensor()
])

augment_transform = transforms.Compose([
    transforms.Resize((ImageHeight, ImageWidth)),
    transforms.RandomRotation(degrees=20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop((ImageHeight, ImageWidth), scale=(0.8, 1.0)),
    transforms.ToTensor()
])

In [13]:

training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=transform,
)

augmented_training_data1 = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=augment_transform,
)

augmented_training_data2 = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=augment_transform,
)

test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform=transform,
)

evaluation_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform=transform,
)

In [14]:
combined_dataset1 = ConcatDataset([training_data, augmented_training_data1])
combined_dataset2 = ConcatDataset([combined_dataset1, augmented_training_data2])
train_dataloader = DataLoader(combined_dataset2, batch_size=batch_size, shuffle=True)

test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
evaluation_dataloader = DataLoader(evaluation_data, batch_size=batch_size, shuffle=False)

In [15]:
for X, y in test_dataloader:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X: torch.Size([64, 3, 200, 200])
Shape of y: torch.Size([64]) torch.int64


In [16]:
class CNN(nn.Module):
    def __init__(self, ImageWidth, ImageHeight, num_classes=102):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(in_features = int(ImageWidth/4)*int(ImageHeight/4)*16, out_features=num_classes)

    def forward(self, x):
          x = self.pool(F.relu(self.conv1(x)))
          x = self.pool(F.relu(self.conv2(x)))
          x = self.fc(x.reshape(x.shape[0], -1))

          return x

model = CNN(ImageWidth, ImageHeight, num_classes=102)

In [17]:
x = torch.randn(32, 3, 200, 200)
y = model(x)
print(y.shape)

torch.Size([32, 102])


In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = CNN(ImageWidth=ImageWidth, ImageHeight=ImageHeight, num_classes=102)
model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr = LearningRate)

cuda


In [19]:
def check_accuracy(loader, model, device):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    accuracy = float(num_correct) / num_samples
    return accuracy

In [20]:
for epoch in range(NumberofEpochs):
  running_loss = 0
  for index, (x,y) in enumerate(train_dataloader):
    x = x.to(device)
    y = y.to(device)

    y_hat = model(x)
    loss = criterion(y_hat, y)
    running_loss += loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f"Epoch {epoch}: loss: {running_loss}")
  eval_accuracy = check_accuracy(evaluation_dataloader, model, device)
  print(f'Evaluation Accuracy: {eval_accuracy * 100:.2f}%')

Epoch 0: loss: 196.6856689453125
Evaluation Accuracy: 14.61%
Epoch 1: loss: 114.85191345214844
Evaluation Accuracy: 24.31%
Epoch 2: loss: 73.69407653808594
Evaluation Accuracy: 25.78%
Epoch 3: loss: 56.41706466674805
Evaluation Accuracy: 25.49%
Epoch 4: loss: 43.79290008544922
Evaluation Accuracy: 27.35%
Epoch 5: loss: 35.019447326660156
Evaluation Accuracy: 26.37%
Epoch 6: loss: 30.344961166381836
Evaluation Accuracy: 28.04%
Epoch 7: loss: 25.660400390625
Evaluation Accuracy: 29.61%
Epoch 8: loss: 21.791732788085938
Evaluation Accuracy: 29.61%
Epoch 9: loss: 20.536895751953125
Evaluation Accuracy: 30.98%
Epoch 10: loss: 16.67045783996582
Evaluation Accuracy: 31.08%
Epoch 11: loss: 14.175838470458984
Evaluation Accuracy: 29.61%
Epoch 12: loss: 12.120975494384766
Evaluation Accuracy: 29.22%
Epoch 13: loss: 13.434870719909668
Evaluation Accuracy: 29.71%
Epoch 14: loss: 11.910536766052246
Evaluation Accuracy: 30.20%
Epoch 15: loss: 10.477455139160156
Evaluation Accuracy: 32.94%
Epoch 16: 

In [22]:
#final test
test_accuracy = check_accuracy(test_dataloader, model, device)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Test Accuracy: 28.87%
